In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Dataset Loading

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NCC/Spring2024/CSCE464/Datasets/Iris.csv')

In [ ]:
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


# Apply the one-hot encoding technique to the species column

In [ ]:
df['Iris-setosa'] = df['Species'].apply(lambda x:1 if x=='Iris-setosa' else 0)
df['Iris-versicolor'] = df['Species'].apply(lambda x:1 if x=='Iris-versicolor' else 0)
df['Iris-virginica'] = df['Species'].apply(lambda x:1 if x=='Iris-virginica' else 0)

In [ ]:
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,5.1,3.5,1.4,0.2,Iris-setosa,1,0,0
1,2,4.9,3.0,1.4,0.2,Iris-setosa,1,0,0
2,3,4.7,3.2,1.3,0.2,Iris-setosa,1,0,0
3,4,4.6,3.1,1.5,0.2,Iris-setosa,1,0,0
4,5,5.0,3.6,1.4,0.2,Iris-setosa,1,0,0
...,...,...,...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica,0,0,1
146,147,6.3,2.5,5.0,1.9,Iris-virginica,0,0,1
147,148,6.5,3.0,5.2,2.0,Iris-virginica,0,0,1
148,149,6.2,3.4,5.4,2.3,Iris-virginica,0,0,1


In [ ]:
# df['Species'] = df['Species'].map({'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2})
# df

# Data Preprocessing

In [ ]:
df = df.drop(['Id', 'Species'], axis=1)
df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Iris-setosa,Iris-versicolor,Iris-virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0,0,1
146,6.3,2.5,5.0,1.9,0,0,1
147,6.5,3.0,5.2,2.0,0,0,1
148,6.2,3.4,5.4,2.3,0,0,1


In [ ]:
x_data = df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
y_data = df[['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']]

In [ ]:
x_data

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [ ]:
y_data

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
145,0,0,1
146,0,0,1
147,0,0,1
148,0,0,1


# Split the dataset

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)

In [ ]:
x_train.shape

(120, 4)

In [ ]:
x_test.shape

(30, 4)

In [ ]:
y_train.shape

(120, 3)

In [ ]:
y_test.shape

(30, 3)

In [ ]:
x_train.reset_index(drop=True)
x_test.reset_index(drop=True)
y_train.reset_index(drop=True)
y_test.reset_index(drop=True)

,Iris-setosa,Iris-versicolor,Iris-virginica
0,0,1,0
1,0,1,0
2,1,0,0
3,0,1,0
4,0,1,0
5,0,1,0
6,1,0,0
7,0,0,1
8,0,0,1
9,0,1,0


In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Softmax Model in TensorFlow

In [ ]:
#Define X, W, and b
x = tf.placeholder(tf.float32, [None, 4])
y = tf.placeholder(tf.float32, [None, 3])
W = tf.Variable(tf.random_normal([4, 3]), name="weight")
b = tf.Variable(tf.random_normal([3]), name="bias")

#Define a hypothesis
hypothesis = tf.nn.softmax(tf.matmul(x, W) + b)

#Define a cost function
cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#Model Training
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range (1001):
  sess.run(optimizer, feed_dict={x: x_train, y: y_train})

  if epoch % 100 == 0:
    _, acc = sess.run([cost, accuracy], feed_dict={x: x_test, y: y_test})
    print("Epoch: {:5}\t Cost: {:.5f}\t ACC: {:.2%}".format(epoch, _, acc))

Epoch:     0	 Cost: 3.86540	 ACC: 10.00%
Epoch:   100	 Cost: 0.45292	 ACC: 76.67%
Epoch:   200	 Cost: 0.35726	 ACC: 86.67%
Epoch:   300	 Cost: 0.30308	 ACC: 86.67%
Epoch:   400	 Cost: 0.26585	 ACC: 93.33%
Epoch:   500	 Cost: 0.23871	 ACC: 93.33%
Epoch:   600	 Cost: 0.21816	 ACC: 93.33%
Epoch:   700	 Cost: 0.20212	 ACC: 96.67%
Epoch:   800	 Cost: 0.18929	 ACC: 96.67%
Epoch:   900	 Cost: 0.17882	 ACC: 96.67%
Epoch:  1000	 Cost: 0.17011	 ACC: 96.67%


# Softmax Model in PyTorch

In [ ]:
x_train, x_test = torch.FloatTensor(x_train), torch.FloatTensor(x_test)
y_train, y_test = torch.FloatTensor(y_train.to_numpy()), torch.FloatTensor(y_test.to_numpy())

In [ ]:
class SoftmaxClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(4, 3)
    self.softmax = nn.Softmax()

  def forward(self, x):
    return self.softmax(self.linear(x))

In [ ]:
model = SoftmaxClassifier()

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

print("====Model Training====")
for epoch in range(1001):
  x = x_train
  y = y_train

  hypothesis = model(x)
  cost = F.cross_entropy(hypothesis, y)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    prediction = hypothesis
    correct_prediction = torch.argmax(prediction, 1) == torch.argmax(y_train, 1)
    accuracy = correct_prediction.float().mean()

    print("Epoch {:4d} / {} Cost: {:.5f} Training ACC: {:.2f}%".format(epoch, 1000, cost.item(), accuracy*100))

====Model Training====
Epoch    0 / 1000 Cost: 1.00490 Training ACC: 75.83%
Epoch  100 / 1000 Cost: 0.77091 Training ACC: 84.17%
Epoch  200 / 1000 Cost: 0.73932 Training ACC: 85.00%
Epoch  300 / 1000 Cost: 0.72274 Training ACC: 85.83%
Epoch  400 / 1000 Cost: 0.71097 Training ACC: 88.33%
Epoch  500 / 1000 Cost: 0.70118 Training ACC: 90.00%
Epoch  600 / 1000 Cost: 0.69233 Training ACC: 90.00%
Epoch  700 / 1000 Cost: 0.68403 Training ACC: 90.00%
Epoch  800 / 1000 Cost: 0.67620 Training ACC: 90.83%
Epoch  900 / 1000 Cost: 0.66891 Training ACC: 92.50%
Epoch 1000 / 1000 Cost: 0.66226 Training ACC: 92.50%
